<a href="https://colab.research.google.com/github/naserjawas/signpy-ml/blob/main/test_phoenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test ASLR System using Phoenix Dataset

Author: Naser Jawas

Created date: 1 August 2024

### Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Files

**Copy pre-trained files from google drive**

The training process is located in *phoenix_recognise_mhi_gloss.ipynb*

In [ ]:
!mkdir -p /content/dataset/RWTHPHOENIXWeather2014/classifiers/
!cp -v /content/drive/MyDrive/classifiers/j10clf1_30[b,f] /content/dataset/RWTHPHOENIXWeather2014/classifiers/
!cp -v /content/drive/MyDrive/classifiers/j10clf1_50[b,f] /content/dataset/RWTHPHOENIXWeather2014/classifiers/

**Copy ngram files**

The ngram files are generated using *phoenix_ngram.py*

In [ ]:
!mkdir -p /content/dataset/RWTHPHOENIXWeather2014/ngram_files/
!cp -v /content/drive/MyDrive/ngram_files/* /content/dataset/RWTHPHOENIXWeather2014/ngram_files/

**Copy segment files**

The segment files are generated using *phoenix_segment.py*



In [ ]:
!mkdir -p /content/dataset/RWTHPHOENIXWeather2014/segment_files/
!cp -v /content/drive/MyDrive/segment_files/* /content/dataset/RWTHPHOENIXWeather2014/segment_files/

### Imports

In [ ]:
import os
import pickle
import glob
import cv2 as cv
import numpy as np

from sklearn.preprocessing import LabelEncoder

### Functions

In [ ]:
def load_clf(drive, glosstype, glossp, mhitype, limitsample):
    rootdir = drive + f"/classifiers/"
    filename = rootdir + f"j{limitsample}clf{glosstype}_{glossp}{mhitype}"
    if not os.path.exists(filename):
        print("Classifier:", filename, "does not exist")
        exit()
    print(filename)
    with open(filename, 'rb') as pf:
        clfobj = pickle.load(pf)
    print(filename, "loaded...")
    print(clfobj)

    return clfobj

In [ ]:
def load_allmcm_video(datadir, dataname, glosstype, glossp, blankframe):
    dataloc = datadir + dataname + f"/1/allmcm{glossp}p*-{glosstype}.png"
    datafiles = sorted(glob.glob(dataloc))
    dataframesori = [cv.imread(filename, cv.IMREAD_GRAYSCALE)
                     for filename in datafiles]
    dataframesori.insert(0, blankframe)
    dataframes = [cv.resize(frame, (iw, ih))
                  for frame in dataframesori]

    return dataframes

### Settings

In [ ]:
drive = "/content/dataset/RWTHPHOENIXWeather2014"

### Main

**Load the pre-trained classifiers:**

In [ ]:
cf30 = load_clf(drive, 1, 30, 'f', 10)
cb30 = load_clf(drive, 1, 30, 'b', 10)

In [ ]:
cf50 = load_clf(drive, 1, 50, 'f', 10)
cb50 = load_clf(drive, 1, 50, 'b', 10)

**Load n-gram objects:**

In [ ]:
n = 3

with open(f"/content/dataset/RWTHPHOENIXWeather2014/ngram_files/phoenix_{n}grams", "rb") as pf:
    ngrams = pickle.load(pf)
pf.close()

**Load filenames:**

In [ ]:
gf, gb, gl =

**Label Encoder:**

In [ ]:
le = LabelEncoder()
le.fit(gl)

**Load segments object:**

In [ ]:
glossp = 30
glosstype = 2

with open(f"/content/dataset/RWTHPHOENIXWeather2014/segments_{glossp}p_{glosstype}", "rb") as pf:
    segments = pickle.load(pf)
pf.close()

**Load test directory:**

In [ ]:
drive = "/content/dataset/RWTHPHOENIXWeather2014"
datadir = drive + "/phoenix2014-release/phoenix-2014-multisigner/features/fullFrame-210x260px/test/"
datanames = sorted(os.listdir(datadir))
if datanames[0] == ".DS_Store":
    datanames.pop(0)
print(len(datanames), "data available...")
iw, ih = 210, 300
blankframe = np.zeros((iw, ih), dtype=np.uint8)

**Load test data:**

In [ ]:
datanamespart = datanames[1:2]

**Recognition proces:**

In [ ]:
finalresults = []
for dataname_i, dataname in enumerate(datanamespart):
    # load original data frames
    dataloc = datadir + dataname + f"/1/*-0.png"
    datafiles = sorted(glob.glob(dataloc))
    lendatafiles = len(datafiles)
    print("(", dataname_i + 1, "/", len(datanamespart),")",
          dataname, "has", lendatafiles, "frames")
    imgori = [cv.imread(filename, cv.IMREAD_COLOR)
              for filename in datafiles]
    dataori = [cv.resize(img, (iw, ih))
               for img in imgori]
    peaks = segments[dataname]
    peaks.insert(0, 0)
    peaks.append(lendatafiles-1)
    allmcm = load_allmcm_video(datadir, dataname, glosstype, glossp, blankframe)
    glossresult1 = []

    glossresult1 = find_path(dataori, dataname, glossresult1, ngrams, peaks, cw, allmcm, cf, cb)
    listgloss = [g[0] for g in glossresult1]
    finalresults.append((dataname, listgloss))

**Save the results:**

In [ ]:
with open('finalresults', 'wb') as pf:
    pickle.dump(finalresults, pf)
pf.close()